In [4]:
data

[[14.0,
  14.0,
  14.0,
  13.0,
  12.0,
  12.0,
  12.0,
  12.0,
  15.0,
  17.0,
  20.0,
  22.0,
  22.0,
  22.0,
  22.0,
  22.0,
  21.0,
  19.0,
  17.0,
  16.0,
  15.0,
  15.0,
  15.0,
  15.0,
  16.0,
  15.0,
  15.0,
  14.0,
  14.0,
  15.0,
  16.0,
  16.0,
  17.0,
  20.0,
  22.0,
  23.0,
  24.0,
  24.0,
  24.0,
  24.0,
  23.0,
  21.0,
  20.0,
  19.0,
  18.0,
  18.0,
  18.0,
  18.0,
  18.0,
  18.0,
  18.0,
  18.0,
  18.0,
  18.0,
  18.0,
  19.0,
  19.0,
  20.0,
  22.0,
  23.0,
  24.0,
  24.0,
  24.0,
  22.0,
  22.0,
  21.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  19.0,
  19.0,
  18.0,
  18.0,
  17.0,
  17.0,
  16.0,
  17.0,
  17.0,
  18.0,
  18.0,
  19.0,
  19.0,
  19.0,
  20.0,
  19.0,
  18.0,
  18.0,
  16.0,
  16.0,
  15.0,
  15.0,
  15.0,
  14.0,
  13.0,
  13.0,
  12.0,
  13.0,
  12.0,
  12.0,
  12.0,
  12.0,
  13.0,
  14.0,
  16.0,
  18.0,
  19.0,
  20.0,
  21.0,
  21.0,
  20.0,
  19.0,
  17.0,
  16.0,
  16.0,
  15.0,
  14.0,
  14.0,
  13.0,
  13.0,
  13.0,
  13.0,
  14.0,
  14.0,


In [2]:
import csv 
import numpy as np
from numpy.linalg import inv
import random
import math
import sys
data = []
# 每一個維度儲存一種污染物的資訊
for i in range(18):
	data.append([])

n_row = 0
text = open('data/train.csv', 'r', encoding='big5') 
row = csv.reader(text , delimiter=",")
for r in row:
    # 第0列沒有資訊
    if n_row != 0:
        # 每一列只有第3-27格有值(1天內24小時的數值)
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0))	
    n_row = n_row+1
text.close()
x = []
y = []
# 每 12 個月
for i in range(12):
    # 一個月取連續10小時的data可以有471筆
    for j in range(471):
        x.append([])
        # 18種污染物
        for t in range(18):
            # 連續9小時
            for s in range(9):
                x[471*i+j].append(data[t][480*i+j+s] )
        y.append(data[9][480*i+j+9])
x = np.array(x)
y = np.array(y)

# add square term
# x = np.concatenate((x,x**2), axis=1)

# add bias
x = np.concatenate((np.ones((x.shape[0],1)),x), axis=1)
w = np.zeros(len(x[0]))
l_rate = 10
repeat = 10000
# use close form to check whether ur gradient descent is good
# however, this cannot be used in hw1.sh 
# w = np.matmul(np.matmul(inv(np.matmul(x.transpose(),x)),x.transpose()),y)
x_t = x.transpose()
s_gra = np.zeros(len(x[0]))

for i in range(repeat):
    hypo = np.dot(x,w)
    loss = hypo - y
    cost = np.sum(loss**2) / len(x)
    cost_a  = math.sqrt(cost)
    gra = np.dot(x_t,loss)
    s_gra += gra**2
    ada = np.sqrt(s_gra)
    w = w - l_rate * gra/ada
    print ('iteration: %d | Cost: %f  ' % ( i,cost_a))
# save model
np.save('model.npy',w)
# read model
w = np.load('model.npy')
test_x = []
n_row = 0
text = open('data/test.csv' ,"r")
row = csv.reader(text , delimiter= ",")

for r in row:
    if n_row %18 == 0:
        test_x.append([])
        for i in range(2,11):
            test_x[n_row//18].append(float(r[i]) )
    else :
        for i in range(2,11):
            if r[i] !="NR":
                test_x[n_row//18].append(float(r[i]))
            else:
                test_x[n_row//18].append(0)
    n_row = n_row+1
text.close()
test_x = np.array(test_x)

# add square term
# test_x = np.concatenate((test_x,test_x**2), axis=1)

# add bias
test_x = np.concatenate((np.ones((test_x.shape[0],1)),test_x), axis=1)
ans = []
for i in range(len(test_x)):
    ans.append(["id_"+str(i)])
    a = np.dot(w,test_x[i])
    ans[i].append(a)

filename = "result/predict.csv"
text = open(filename, "w+")
s = csv.writer(text,delimiter=',',lineterminator='\n')
s.writerow(["id","value"])
for i in range(len(ans)):
    s.writerow(ans[i]) 
text.close()

iteration: 0 | Cost: 27.071215  
iteration: 1 | Cost: 50528.584594  
iteration: 2 | Cost: 27.066937  
iteration: 3 | Cost: 14.762751  
iteration: 4 | Cost: 14.541508  
iteration: 5 | Cost: 14.329953  
iteration: 6 | Cost: 14.127694  
iteration: 7 | Cost: 13.934354  
iteration: 8 | Cost: 13.749566  
iteration: 9 | Cost: 13.572976  
iteration: 10 | Cost: 13.404239  
iteration: 11 | Cost: 13.243023  
iteration: 12 | Cost: 13.089005  
iteration: 13 | Cost: 12.941874  
iteration: 14 | Cost: 12.801328  
iteration: 15 | Cost: 12.667076  
iteration: 16 | Cost: 12.538838  
iteration: 17 | Cost: 12.416343  
iteration: 18 | Cost: 12.299329  
iteration: 19 | Cost: 12.187546  
iteration: 20 | Cost: 12.080752  
iteration: 21 | Cost: 11.978715  
iteration: 22 | Cost: 11.881210  
iteration: 23 | Cost: 11.788024  
iteration: 24 | Cost: 11.698951  
iteration: 25 | Cost: 11.613793  
iteration: 26 | Cost: 11.532362  
iteration: 27 | Cost: 11.454477  
iteration: 28 | Cost: 11.379965  
iteration: 29 | Cost: